In [ ]:
%pip install nbstripout
!nbstripout --install

REPO_URL="https://github.com/jo5huar3/UH-Insure-NSA.git"
BRANCH="main"
REPO="UH-Insure-NSA"
# optional: start clean
!rm -rf "$REPO"

# clone (shallow) and enter it
!git clone --branch "$BRANCH" "$REPO_URL"
%cd /content/UH-Insure-NSA


!if [ -f requirements.txt ]; then pip install -r requirements.txt; fi
!if [ -f pyproject.toml ]; then pip install -e .; fi  # for packages

Cloning into 'UH-Insure-NSA'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 66 (delta 27), reused 40 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 48.07 KiB | 5.34 MiB/s, done.
Resolving deltas: 100% (27/27), done.
/content/UH-Insure-NSA


In [ ]:
from model.test import test
test()

Hello World!


In [ ]:
!pip -q install -U huggingface_hub hf_transfer
!export HF_HUB_ENABLE_HF_TRANSFER=1

import os
from google.colab import userdata
from huggingface_hub import login, whoami

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # mitigate fragmentation
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN, add_to_git_credential=True)  # also sets Git creds for LFS

print("Logged in as:", whoami(token=HF_TOKEN)["name"])

Logged in as: j05hr3d


In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'code-llm-finetuning'
!gcloud config set project {project_id}

GCS_PATH = "gs://code-llm-fine-tuning-security-analytics/data/source_code.jsonl"
LOCAL_PATH = "/tmp/source_code.jsonl"

!gsutil ls {GCS_PATH}
!gsutil -m cp {GCS_PATH} {LOCAL_PATH}

!ls -lh {LOCAL_PATH}
!head -n 2 {LOCAL_PATH}

#!gsutil cp gs://code-llm-fine-tuning-security-analytics/data/source_code.jsonl /tmp/source_code.jsonl
#!cat /tmp/source_code.jsonl

Updated property [core/project].
gs://code-llm-fine-tuning-security-analytics/data/source_code.jsonl
Copying gs://code-llm-fine-tuning-security-analytics/data/source_code.jsonl...
- [1/1 files][  3.4 KiB/  3.4 KiB] 100% Done                                    
Operation completed over 1 objects/3.4 KiB.                                      
-rw-r--r-- 1 root root 3.4K Sep 13 16:14 /tmp/source_code.jsonl
{"filename": "cryptol/examples/Karatsuba.cry", "content": "/*\n * Copyright (c) 2017 Galois, Inc.\n * Distributed under the terms of the BSD3 license (see LICENSE file)\n *\n * This module defines a Karatsuba multiplier, which is polymorphic over the\n * bitwidth of the constituent limbs and the size of the inputs.\n *\n * Note the trick of using `make_atleast` in the recursive case to teach the\n * typecheker that the current bitwidth is long enough for the split multiplier\n * to be sensible.  The minimum width for which the splitting recursive case\n * works out happens to be 6.  Thi

In [ ]:
!pip install -q transformers datasets peft bitsandbytes flash-attn

MODEL = "Qwen/Qwen2.5-Coder-7B"  # Model checkpoint on the Hugging Face Hub
DATASET = "/tmp/source_code.jsonl"
DATA_COLUMN = "content"  # Column name containing the code content

SEQ_LENGTH = 2048  # Sequence length

# Training arguments
MAX_STEPS = 1#2000  # max_steps
BATCH_SIZE = 1  # batch_size
GR_ACC_STEPS = 16  # gradient_accumulation_steps
LR = 5e-4  # learning_rate
LR_SCHEDULER_TYPE = "cosine"  # lr_scheduler_type
WEIGHT_DECAY = 0.01  # weight_decay
NUM_WARMUP_STEPS = 30  # num_warmup_steps
EVAL_FREQ = 100  # eval_freq
SAVE_FREQ = 100  # save_freq
LOG_FREQ = 25  # log_freq
OUTPUT_DIR = "peft-FT-2.5-Coder-7b"  # output_dir
BF16 = True  # bf16
FP16 = False  # no_fp16

# FIM trasformations arguments
FIM_RATE = 0.5  # fim_rate
FIM_SPM_RATE = 0.5  # fim_spm_rate

# LORA
LORA_R = 8  # lora_r
LORA_ALPHA = 32  # lora_alpha
LORA_DROPOUT = 0.05  # lora_dropout
LORA_TARGET_MODULES = "c_proj,c_attn,q_attn,c_fc,c_proj"  # lora_target_modules

# bitsandbytes config
USE_NESTED_QUANT = True  # use_nested_quant
BNB_4BIT_COMPUTE_DTYPE = "bfloat16"  # bnb_4bit_compute_dtype

SEED = 0

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    logging,
    set_seed,
    BitsAndBytesConfig,
)

set_seed(SEED)

In [ ]:
from datasets import load_dataset

DATASET = LOCAL_PATH
DATA_COLUMN = "content"

dataset = load_dataset("json", data_files={"train": DATASET})
train_data = dataset["train"]

# quick sanity check: make sure the field exists
print(train_data[0].keys())
assert DATA_COLUMN in train_data.column_names, f"Missing '{DATA_COLUMN}' in JSONL!"


Generating train split: 0 examples [00:00, ? examples/s]

dict_keys(['filename', 'content'])


In [ ]:
import torch
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

def chars_token_ratio(dataset, tokenizer, data_column, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """

    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        total_characters += len(example[data_column])
        total_tokens += len(tokenizer(example[data_column]).tokens())

    return total_characters / total_tokens


chars_per_token = chars_token_ratio(train_data, tokenizer, DATA_COLUMN)
print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")


  0%|          | 1/400 [00:00<00:01, 259.37it/s]

The character to token ratio of the dataset is: 3.19


In [ ]:
from model.dataset.util import *

In [ ]:
from model.dataset.constantLengthDataset import ConstantLengthDataset


train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        infinite=True,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)
eval_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        #valid_data,
        infinite=False,
        seq_length=SEQ_LENGTH,
        chars_per_token=chars_per_token,
        content_field=DATA_COLUMN,
        fim_rate=FIM_RATE,
        fim_spm_rate=FIM_SPM_RATE,
        seed=SEED,
)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.tuners.lora import LoraLayer

load_in_8bit = False

# 4-bit quantization
compute_dtype = getattr(torch, BNB_4BIT_COMPUTE_DTYPE)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=USE_NESTED_QUANT,
)

device_map = "auto"#{"": 0}

model = AutoModelForCausalLM.from_pretrained(
        MODEL,
        load_in_8bit=load_in_8bit,
        quantization_config=bnb_config,
        device_map=device_map,
        use_cache=False,  # We will be using gradient checkpointing
        trust_remote_code=True,
        attn_implementation="flash_attention_2",
)

model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
blk = model.model.layers[0]           # Llama/Qwen-style
print("ATTN:", blk.self_attn)         # has q_proj, k_proj, v_proj, o_proj
print("MLP:", blk.mlp)
target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

ATTN: Qwen2Attention(
  (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
  (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
  (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
  (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
)
MLP: Qwen2MLP(
  (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
  (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
  (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
  (act_fn): SiLU()
)


In [ ]:
# Set up lora
peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


In [ ]:
train_data.start_iteration = 0


training_args = TrainingArguments(
    output_dir=f"j05hr3d/{OUTPUT_DIR}",
    dataloader_drop_last=True,
    eval_strategy="steps",
    save_strategy="steps",
    max_steps=MAX_STEPS,
    eval_steps=EVAL_FREQ,
    save_steps=SAVE_FREQ,
    logging_steps=LOG_FREQ,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_steps=NUM_WARMUP_STEPS,
    gradient_accumulation_steps=GR_ACC_STEPS,
    gradient_checkpointing=True,
    fp16=FP16,
    bf16=BF16,
    weight_decay=WEIGHT_DECAY,
    push_to_hub=True,
    include_tokens_per_second=True,
    report_to=[]
)

In [ ]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

print("Training...")
trainer.train()

Training...


Step,Training Loss,Validation Loss


TrainOutput(global_step=1, training_loss=0.8815891146659851, metrics={'train_runtime': 19.9819, 'train_samples_per_second': 0.801, 'train_steps_per_second': 0.05, 'train_tokens_per_second': 1639.887, 'total_flos': 1394108836872192.0, 'train_loss': 0.8815891146659851, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...t-FT-2.5-Coder-7b/training_args.bin: 100%|##########| 5.78kB / 5.78kB            

  ...-Coder-7b/adapter_model.safetensors:  52%|#####1    | 41.8MB / 80.8MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/j05hr3d/peft-FT-2.5-Coder-7b/commit/f533aa373fae315bcdc01129dde3a8ee69fad810', commit_message='End of training', commit_description='', oid='f533aa373fae315bcdc01129dde3a8ee69fad810', pr_url=None, repo_url=RepoUrl('https://huggingface.co/j05hr3d/peft-FT-2.5-Coder-7b', endpoint='https://huggingface.co', repo_type='model', repo_id='j05hr3d/peft-FT-2.5-Coder-7b'), pr_revision=None, pr_num=None)

In [37]:
# Sanitize the most-recently modified notebook in your repo
import os, glob, time
import nbformat as nbf

# Detect repo root (prefers CWD if it's a git repo; falls back to your repo path)
candidates = [os.getcwd(), "/content/UH-Insure-NSA", "/content"]
repo_root = next(
    (p for p in candidates if os.path.isdir(p) and (os.path.isdir(os.path.join(p, ".git")) or p.endswith("UH-Insure-NSA"))),
    "/content"
)

# Find the newest .ipynb
paths = glob.glob(os.path.join(repo_root, "**", "*.ipynb"), recursive=True)
if not paths:
    raise SystemExit(f"No notebooks found under {repo_root}")

target = max(paths, key=os.path.getmtime)

# Remove notebook- and cell-level widgets metadata
nb = nbf.read(target, as_version=nbf.NO_CONVERT)
touched = False

if isinstance(nb.metadata, dict) and "widgets" in nb.metadata:
    nb.metadata.pop("widgets", None); touched = True

for cell in nb.cells:
    md = cell.get("metadata", {})
    if isinstance(md, dict) and "widgets" in md:
        md.pop("widgets", None); touched = True

if touched:
    nbf.write(nb, target)

print("Sanitized:", os.path.relpath(target, repo_root))
print("Timestamp:", time.ctime(os.path.getmtime(target)))
print("Repo root:", repo_root)

NB_PATH = "/content/UH-Insure-NSA/notebooks/codeLLMFT.ipynb"  # <- your file
nb = nbf.read(NB_PATH, as_version=nbf.NO_CONVERT)
# ...same cleanup...
nbf.write(nb, NB_PATH)



Sanitized: notebooks/codeLLMFT.ipynb
Timestamp: Sat Sep 13 16:18:05 2025
Repo root: /content/UH-Insure-NSA
